In [ ]:
import PIL
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import PIL
import tensorflow as tf
from PIL import ImageShow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator    
from tensorflow.keras.models import Sequential

In [16]:
my_data = pd.read_csv('../datasets/annotated_honeybee/bee_data.csv')
dataset_path = '../datasets/annotated_honeybee/bee_imgs2'
data_dir = pathlib.Path(dataset_path)

In [17]:
image_count = len(list(data_dir.glob('*/*.png')))
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*')])
CLASS_NAMES

array(['few_var', 'varroa', 'healthy'], dtype='<U7')

In [19]:
image_count = len(list(data_dir.glob('few_var/*.png')))
image_count

579

In [20]:
batch_size = 54
img_height = 150
img_width = 150
datagen_params = {
    "horizontal_flip"  : True,    
    "vertical_flip"    : True,
    "rotation_range"   : 60,
    "brightness_range" : [.7, 1.]
}

In [23]:
def normalize(image):
    '''
    Returns the given np.array image rescaled and normalized to be between -.5 and .5  
    Source: https://www.jeremyjordan.me/batch-normalization/
    '''
    return (image/255. - 0.5)

In [24]:
train_datagen = ImageDataGenerator(
   horizontal_flip  = datagen_params.get("horizontal_flip") or False,
   vertical_flip    = datagen_params.get("vertical_flip") or False,
   rotation_range   = datagen_params.get("rotation_range") or False,
   brightness_range = datagen_params.get("brightness_range"),
   preprocessing_function = normalize
)
train_generator = train_datagen.flow_from_directory(
       dataset_path,
       target_size=(img_height, img_width),
       batch_size=batch_size,

)
test_datagen = ImageDataGenerator(
   validation_split=0.3,
   preprocessing_function = normalize
)
validation_generator = test_datagen.flow_from_directory(
       dataset_path,
       subset="validation",
       target_size=(img_height, img_width),
       batch_size=batch_size,
)

Found 4443 images belonging to 3 classes.
Found 1331 images belonging to 3 classes.


In [27]:
# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=0.3,
#     subset="validation",
#     seed=123,
#     image_size=(img_height, img_width),
#     batch_size=batch_size
# )
# type(val_ds)
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")
#         plt.show() #NEEDED. Will not display images unless this is added.
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
# normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]

In [28]:
num_classes = 3
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(64, (4,4), padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),
    layers.Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),
    layers.Conv2D(32, (3,3), padding='same', activation='relu',
#                   kernel_initializer ='he_normal', input_shape=(img_height, img_width, 3)
     ),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])
# model.compile(
#     optimizer='adam',
#     metrics=['accuracy'],
#     loss = 'categorical_crossentropy',
# )
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer='Adam', metrics=['accuracy'])

NameError: name 'regularizers' is not defined

In [29]:
epochs=50
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs
)
# history = model.fit(
#         train_generator,

#         validation_data=validation_generator,
#    epochs=50,
# )

NameError: name 'model' is not defined

In [30]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Testing Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Testing Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Testing Loss')
plt.legend(loc='upper right')
plt.title('Training and Testing Loss')
plt.show()

NameError: name 'history' is not defined

In [31]:
#Bees Infected with Varroa Mites
varroa_infected_1 = "../datasets/annotated_honeybee/predictions/varroa_infected_1.png"
varroa_infected_2 = "../datasets/annotated_honeybee/predictions/varroa_infected_2.png"
varroa_infected_3 = "../datasets/annotated_honeybee/predictions/varroa_infected_3.png"
varroa_infected_4 = "../datasets/varroa_test_set/has_varroa/001_001.jpeg"
varroa_infected_5 = "../datasets/varroa_test_set/has_varroa/001_002.jpg"
varroa_infected_6 = "../datasets/varroa_test_set/has_varroa/001_003.png"
#Healthy Bees
healthy_bee_from_data  = "../datasets/annotated_honeybee/bee_imgs2/healthy/001_054.png"
healthy_bee_side = "../datasets/annotated_honeybee/predictions/healthy_bee_side_profile.png"
healthy_bee_top = "../datasets/annotated_honeybee/predictions/healthy_bee_top_view.png"

In [32]:
img = keras.preprocessing.image.load_img(
    varroa_infected_1, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


img = keras.preprocessing.image.load_img(
    varroa_infected_2, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


img = keras.preprocessing.image.load_img(
    varroa_infected_3, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

img = keras.preprocessing.image.load_img(
    varroa_infected_4, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

img = keras.preprocessing.image.load_img(
    varroa_infected_5, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

img = keras.preprocessing.image.load_img(
    varroa_infected_6, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


print('=======================================================')
img = keras.preprocessing.image.load_img(
    healthy_bee_from_data, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
img = keras.preprocessing.image.load_img(
    healthy_bee_top, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
     "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


img = keras.preprocessing.image.load_img(
    healthy_bee_side, target_size=(img_height, img_width)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "{} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

NameError: name 'model' is not defined

In [33]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_ds.class_names
#Prints how many classes are in the new dataset
print(class_names)

AttributeError: module 'tensorflow.keras.preprocessing' has no attribute 'image_dataset_from_directory'